In a [previous post](), I introduced xarray with some simple manipulation and data plotting. Here I'm going to do some more manipulation. A synopsis of the post follows.

#### GOAL:
The ultimate goal here is to create new datasets, one for band, that aggregate results across experiments so as to facilitate inter-experiment comparisons.

#### HOW: 
I will load netCDF files from a number of Monte-Carlo uncertainty experiments, among which the source of the uncertainty differs; Lt (sensor noise), wind, pressure, relative humidity, all the above.
At the end of this post, I will have 6 files, one per visible SeaWiFS visible band
containing one 3D array where dimensions are latitude, longitude, experiment.

#### WHY: 
I'm doing this to create an interactive visualization (*cf.* [next post]()) using GeoViews, where the goal is to compare, band-wise, the cross-experiment results.

As usual, start with some imports...

In [1]:
import xarray as xr
import os
import glob

Now I set up some file path logic to avoid rewriting full file paths.

In [2]:
dataDir = '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/'
expDirs = ['Lt', 'AllAnc_Lt', 'Pressure', 'RH', 'WindSpeed']
outDir = 'Synthesis'
fpattern = 'S20031932003196.L3m_4D_SU*.nc'
fpaths = [glob.glob(os.path.join(dataDir, expDir, fpattern))[0] for expDir in expDirs]

This is the list of paths where files containing experimental resulst are contained:

In [3]:
fpaths

['/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/Lt/S20031932003196.L3m_4D_SU50.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/AllAnc_Lt/S20031932003196.L3m_4D_SU46.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/Pressure/S20031932003196.L3m_4D_SU49.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/RH/S20031932003196.L3m_4D_SU47.nc',
 '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/DATA/AncillaryMC/WindSpeed/S20031932003196.L3m_4D_SU48.nc']

xarray has a nifty feature that allows opening multiple datasets, and automatically concatenating matching (by name and dimension) arrays, with the option of naming the thus newly created dimension. You will need [dask](http://dask.pydata.org/en/latest/) installed for his. In our case, this is *'experiment'*. The line below opens what will end up being a temporary xarray Dataset.

In [4]:
allData = xr.open_mfdataset(fpaths, concat_dim='experiment')

Now I label the *experiment* dimension with the appropriate experiment names. Here,  the concatenation direction reflects the order in which the file paths are specified, which in turn is that in which the experiment names are listed in expDirs.

In [5]:
allData.coords['experiment'] = expDirs

In [6]:
bands = [412, 443, 490, 510, 555, 670]
for band in bands:
    foutpath = os.path.join(dataDir, outDir, '%s%d%s' %(fpattern.split('SU')[0],
                                                        band, '.nc'))
    if not os.path.exists(os.path.dirname(foutpath)):
        os.makedirs(os.path.dirname(foutpath))
    data = allData.data_vars['Rrs_unc_%d' % band]
    data.name='rrs_unc_%d' % band
    dsData = data.to_dataset()
    dsData.to_netcdf(path=foutpath, engine='netcdf4')

I didn't use a *with* context which would have closed the temporary *allData* dataset

In [7]:
allData.close()

Verify that all the files are where they should be; in the Syntesis directory

In [8]:
os.listdir(os.path.dirname(foutpath))

['S20031932003196.L3m_4D_490.nc',
 'S20031932003196.L3m_4D_510.nc',
 'S20031932003196.L3m_4D_412.nc',
 'S20031932003196.L3m_4D_555.nc',
 'S20031932003196.L3m_4D_670.nc',
 'S20031932003196.L3m_4D_443.nc']

Success!! 
That's it. In the [next post]() I will use GeoViews, HoloViews, with a Matplotlib backend to create some nice, but more importantly, intuitive and hopefully useful visualizations.